In [10]:
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import logging
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import math

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

datapath = 'data\\reduced_temperature_data.xlsx'

X_data = np.array(pd.read_excel(datapath, sheet_name = 'boundary_condition'))
temperature_data = np.transpose(np.array(pd.read_excel(datapath, sheet_name = 'temperature')))

(9, 2)
(9, 5760)


In [8]:
training_condition = (X_data[:,1] == 10)|(X_data[:,1] == 12.5)|(X_data[:,1] == 15)|(X_data[:,1] == 17.5)|(X_data[:,1] == 20)
training_row = np.where(training_condition)[0]

row = training_row
x_train_data = X_data[row,:]
temperature = temperature_data[row,:]

A = temperature
U,sigma,V = np.linalg.svd(A)
S = np.zeros(np.shape(A))
for i in range(sigma.shape[0]):
    S[i,i] = sigma[i]
An = np.dot(U,S)

A_ROM = np.zeros(np.shape(A))
Modes_set = [1,2,3,4,5,6,7,8,9]

An_mode = np.zeros(np.shape(An))
V_mode = np.zeros(np.shape(V))

error = []

In [9]:
import time
from scipy.interpolate import Rbf
start =time.time()

mode_num = 5
y_train_data = An[:,:mode_num]

rbf_interpolators = []

for i in range(y_train_data.shape[1]):
    rbf_interpolator = Rbf(x_train_data[:, 0], x_train_data[:, 1], y_train_data[:, i], function='multiquadric')
    rbf_interpolators.append(rbf_interpolator)

x_test = np.delete(X_data,row,0)
y_test = np.delete(temperature_data,row,0)

An_pred = np.zeros((x_test.shape[0],5))

for i in range(x_test.shape[0]):
    new_point = x_test[i]
    new_Y = np.array([rbf(new_point[0], new_point[1]) for rbf in rbf_interpolators])
    An_pred[i] = new_Y


y_pred = np.dot(An_pred, V[:mode_num])

NRMSE = []
for i in range(x_test.shape[0]):
    NRMSE.append((mean_squared_error(y_test[i,:], y_pred[i,:]))**0.5/(np.max(y_test[i,:])-np.min(y_test[i,:])))

MAE = []
for i in range(x_test.shape[0]):
    MAE.append(mean_absolute_error(y_test[i,:], y_pred[i,:]))

print(np.mean(MAE),np.std(MAE))
print(np.round(MAE,2))
print(np.mean(NRMSE))
print(np.round(NRMSE,2))

df = pd.DataFrame(y_pred)

df.to_excel('RBF.xlsx', index=False, header=False, engine='openpyxl')
end=time.time()
print('Running time: %s Seconds'%(end-start))

5.552101357048983 3.306671385700064
[ 1.75  4.77  8.09 11.49  1.19  3.61  6.76 10.18  1.1   3.21  6.14  9.6
  1.08  3.33  6.2   9.47  2.38  3.96  6.76  9.97]
0.3839865287770421
[0.09 0.24 0.4  0.57 0.07 0.21 0.39 0.58 0.07 0.22 0.41 0.64 0.09 0.27
 0.49 0.75 0.18 0.39 0.66 0.96]
Running time: 0.025074005126953125 Seconds
